Doubles Competitor Rankings

In [8]:
import requests

url = "https://api.sportradar.com/tennis/trial/v3/en/double_competitors_rankings.json?api_key=Wsd3stMv53a8qB99rszTUTOScJTU2EhP5V93nlfy"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.text)

{"generated_at":"2025-02-01T14:27:50+00:00","rankings":[{"type_id":2,"name":"ATP","year":2025,"week":5,"gender":"men","competitor_rankings":[{"rank":1,"movement":0,"points":7530,"competitions_played":22,"competitor":{"id":"sr:competitor:49363","name":"Pavic, Mate","country":"Croatia","country_code":"HRV","abbreviation":"PAV"}},{"rank":1,"movement":0,"points":7530,"competitions_played":22,"competitor":{"id":"sr:competitor:51836","name":"Arevalo-Gonzalez, Marcelo","country":"El Salvador","country_code":"SLV","abbreviation":"ARE"}},{"rank":3,"movement":10,"points":7075,"competitions_played":31,"competitor":{"id":"sr:competitor:637970","name":"Patten, Henry","country":"Great Britain","country_code":"GBR","abbreviation":"PAT"}},{"rank":4,"movement":10,"points":7055,"competitions_played":28,"competitor":{"id":"sr:competitor:14898","name":"Heliovaara, Harri","country":"Finland","country_code":"FIN","abbreviation":"HEL"}},{"rank":5,"movement":-2,"points":6475,"competitions_played":21,"competit

In [9]:
double=response.json()

In [10]:
double_rank=[]

In [11]:
for dblcomp in double["rankings"]:
    for ranks in dblcomp.get( "competitor_rankings",[]):
        rank_info={
            "rank":ranks["rank"],
            "movement":ranks["movement"],
            "points":ranks["points"],
            "competitions_played":ranks["competitions_played"],
            "competitor_id":ranks["competitor"].get("id")
        }
        double_rank.append(rank_info)
  

In [12]:
import pandas as pd
df5=pd.DataFrame(double_rank)


In [13]:
df5.to_csv("Competitor_Rankings.csv",index=False)

Competitors Table

In [14]:
comptr=[]

In [15]:
for comp in double["rankings"]:
    for ranks in comp.get( "competitor_rankings",[]):
        compx_info={
            "competitor_id":ranks["competitor"].get("id"),
            "name":ranks["competitor"].get("name"),
            "country":ranks["competitor"].get("country"),
            "country_code":ranks["competitor"].get("country_code"),
            "abbreviation":ranks["competitor"].get("abbreviation")
        }
        comptr.append(compx_info)
      


In [16]:
import pandas as pd
df6=pd.DataFrame(comptr)

In [17]:
df6.fillna("NULL",inplace=True)

In [18]:
df6.to_csv("Competitors.csv",index=False)

In [1]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [50]:
!python.exe -m pip install --upgrade pip

In [19]:
import mysql.connector

connection= mysql.connector.connect(
 host= "localhost",
 user= "root",
 password="5666",
 database="tennis_new"
)

print(connection)
mycursor = connection.cursor()

In [20]:
mycursor.execute("""
CREATE TABLE Competitors (
    competitor_id VARCHAR(50) PRIMARY KEY,
    name VARCHAR(100),
    country VARCHAR(100),
    country_code CHAR(10),
    abbreviation VARCHAR(10));""")
connection.commit()
print("table_5 competitors created column added")


ProgrammingError: 1050 (42S01): Table 'competitors' already exists

In [21]:
mycursor.execute("""
CREATE TABLE Competitor_Rankings (
    rank_id SERIAL PRIMARY KEY, 
    ranking INT,
    movement INT,
    points INT,
    competitions_played INT,
    competitor_id VARCHAR(50),
    FOREIGN KEY (competitor_id) REFERENCES Competitors(competitor_id));""")
connection.commit()
print("table_6 competitor_ranking created column added")


ProgrammingError: 1050 (42S01): Table 'competitor_rankings' already exists

In [22]:
insert = """ 
insert into Competitors (competitor_id, name, country, country_code, abbreviation)
values (%s, %s, %s, %s, %s) ON DUPLICATE KEY UPDATE name=VALUES(name)
"""

for index, row in df6.iterrows():
    mycursor.execute(insert, (
        row['competitor_id'], 
        row['name'], 
        row['country'], 
        row['country_code'], 
        row['abbreviation']
    )
      )
connection.commit()
print("table_6 competitor_ranking inserted values in column wise ")

table_6 competitor_ranking inserted values in column wise 


In [26]:
insert_query="""
insert into Competitor_Rankings (ranking , movement , points , competitions_played , competitor_id) 
values (%s,%s,%s,%s,%s)
"""
for index,row in df5.iterrows():
    mycursor.execute(insert_query,(
        row['rank'],
        row['movement'],
        row['points'],
        row['competitions_played'],
        row['competitor_id']
    )
      )
connection.commit()
print("table_5 competitor_ranking inserted values in column wise ")

table_5 competitor_ranking inserted values in column wise 


In [27]:
#1)Get all competitors with their rank and points.
mycursor.execute("""SELECT Competitors.name,Competitor_Rankings.ranking,Competitor_Rankings.points
                FROM Competitors inner join Competitor_Rankings ON 
                 Competitors.competitor_id=Competitor_Rankings.competitor_id""")
out=mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out,headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

+--------------------------------------+-----------+----------+
| name                                 |   ranking |   points |
|--------------------------------------+-----------+----------|
| Pavic, Mate                          |         1 |     7530 |
| Arevalo-Gonzalez, Marcelo            |         1 |     7530 |
| Patten, Henry                        |         3 |     7075 |
| Heliovaara, Harri                    |         4 |     7055 |
| Thompson, Jordan                     |         5 |     6475 |
| Krawietz, Kevin                      |         6 |     6240 |
| Granollers, Marcel                   |         7 |     6170 |
| Zeballos, Horacio                    |         7 |     6170 |
| Putz, Tim                            |         9 |     6150 |
| Bolelli, Simone                      |        10 |     5990 |
| Vavassori, Andrea                    |        11 |     5990 |
| Mektic, Nikola                       |        12 |     5750 |
| Purcell, Max                         |

In [28]:
#Find competitors ranked in the top 5
mycursor.execute("""SELECT Competitors.name,Competitor_Rankings.ranking
                FROM Competitors inner join Competitor_Rankings ON 
                 Competitors.competitor_id=Competitor_Rankings.competitor_id ORDER BY ranking LIMIT 5""")
out=mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out,headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

+---------------------------+-----------+
| name                      |   ranking |
|---------------------------+-----------|
| Pavic, Mate               |         1 |
| Arevalo-Gonzalez, Marcelo |         1 |
| Siniakova, Katerina       |         1 |
| Routliffe, Erin           |         2 |
| Patten, Henry             |         3 |
+---------------------------+-----------+


In [29]:
#3)List competitors with no rank movement (stable rank)
mycursor.execute("""SELECT Competitors.name, Competitor_Rankings.ranking
FROM Competitors
INNER JOIN Competitor_Rankings
    ON Competitors.competitor_id = Competitor_Rankings.competitor_id
WHERE Competitor_Rankings.ranking = Competitor_Rankings.ranking
ORDER BY ranking""")
out=mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out,headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

+--------------------------------------+-----------+
| name                                 |   ranking |
|--------------------------------------+-----------|
| Pavic, Mate                          |         1 |
| Arevalo-Gonzalez, Marcelo            |         1 |
| Siniakova, Katerina                  |         1 |
| Routliffe, Erin                      |         2 |
| Patten, Henry                        |         3 |
| Townsend, Taylor                     |         3 |
| Heliovaara, Harri                    |         4 |
| Dabrowski, Gabriela                  |         4 |
| Thompson, Jordan                     |         5 |
| Ostapenko, Jelena                    |         5 |
| Krawietz, Kevin                      |         6 |
| Errani, Sara                         |         6 |
| Granollers, Marcel                   |         7 |
| Zeballos, Horacio                    |         7 |
| Paolini, Jasmine                     |         7 |
| Hsieh, Su-Wei                        |      

In [30]:
#4)Get the total points of competitors from a specific country (e.g., Croatia)
mycursor.execute("""SELECT  country, sum(points) AS sum_of_points FROM (SELECT Competitors.country,Competitor_Rankings.points
                FROM Competitors inner join Competitor_Rankings ON 
                 Competitors.competitor_id=Competitor_Rankings.competitor_id) AS NEWTABLE WHERE country = 'USA' """)
out=mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out,headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

+-----------+-----------------+
| country   |   sum_of_points |
|-----------+-----------------|
| USA       |           89711 |
+-----------+-----------------+


In [31]:
#5)Count the number of competitors per country
mycursor.execute("SELECT country,COUNT(name) AS Total_competitor from Competitors group by country order by Total_competitor DESC")
out=mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out,headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

+--------------------------+--------------------+
| country                  |   Total_competitor |
|--------------------------+--------------------|
| USA                      |                 92 |
| Neutral                  |                 66 |
| France                   |                 56 |
| Australia                |                 53 |
| Great Britain            |                 52 |
| Japan                    |                 49 |
| Italy                    |                 43 |
| Czechia                  |                 40 |
| Netherlands              |                 34 |
| China                    |                 33 |
| Germany                  |                 31 |
| Argentina                |                 31 |
| Spain                    |                 28 |
| India                    |                 27 |
| Brazil                   |                 26 |
| Switzerland              |                 20 |
| Ukraine                  |                 19 |


In [32]:
#6Find competitors with the highest points in the current week
mycursor.execute("SELECT max(points) AS APEX_POINT from Competitor_Rankings")
out=mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out,headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

+--------------+
|   APEX_POINT |
|--------------|
|        10750 |
+--------------+
